<a href="https://colab.research.google.com/github/EtiPeretz/Weighted-Random-Forest-implementation/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import *
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from  sklearn.base import BaseEstimator
import random
from sklearn.metrics import zero_one_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from random import randrange





In [ ]:
!git clone https://github.com/fe1493/Fashion_DataSet

fatal: destination path 'Fashion_DataSet' already exists and is not an empty directory.


# Section 1: Weighted Random Forest implementation

In the first part, you are requested to implement a variation of Random Forest, which we will call "weighted" random forest (WRF vs. RF).


In [ ]:
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
import numpy as np
from sklearn.tree import *
from sklearn.metrics import mean_squared_error, zero_one_loss
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
import itertools as it
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
import random
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


class WRF(BaseEstimator):
    # init a WRF classifier with the following parameters:
    def __init__(self, n_trees=100, max_depth=5, n_features=None, type="cat", weight_type="div"):
        '''
          init a WRF classifier with the following parameters:
          n_trees: the number of trees to use.
          max_depth: the depth of each tree (will be passed along to DecisionTreeClassifier/DecisionTreeRegressor).
          n_features: the number of features to use for every split. The number should be given to DecisionTreeClassifier/Regressor as max_features.
          type: "cat" for categorization and "reg" for regression.
          weight_type: the tree weighting technique. 'div' for 1/error and 'sub' for 1-error.
        '''
        # n_trees: the number of trees to use
        self.n_trees = n_trees
        # max_depth: the depth of each tree
        self.max_depth = max_depth
        # n_features: the number of features to use for every split.
        self.n_features = n_features
        self.type = type
        # weight_type: the tree weighting technique.
        self.weight_type = weight_type



    def fit(self, X, y):
        '''
          fit the classifier for the data X with response y.
        '''
        self.trees = []
        self.weights = []
        # <Your Code if needed>
        for n in range(self.n_trees):
            tree = self.build_tree()
            self.trees.append(tree)
            X_tree, y_tree, X_oob, y_oob = self.bootstrap(X, y)
            tree.fit(X_tree, y_tree)
            weight = self.calculate_weight(tree, X_oob, y_oob)
            self.weights.append(weight)

        # Normalize the weights so they sum to 1
        temp = np.array(self.weights - min(self.weights))
        self.weights = temp / sum(temp)  # Sum total to 1.0
        #print("sum of weight is: ", sum(self.weights))
        #self.weights = normalize(self.weights)

    def build_tree(self):
        tree = None
        if self.type == "cat":
            tree = DecisionTreeClassifier(max_depth=self.max_depth, max_features=self.n_features)
        else:
            tree = DecisionTreeRegressor(max_depth=self.max_depth, max_features=self.n_features)
        return tree

    def bootstrap(self, X, y):
        '''
          This method creates a bootstrap of the dataset (uniformly sample len(X) samples from X with repetitions).
          It returns X_tree, y_tree, X_oob, y_oob.
          X_tree, y_tree are the bootstrap collections for the given X and y.
          X_oob, y_oob are the out of bag remaining instances (the ones that were not sampled as part of the bootstrap)
        # '''
        X_tree = []
        y_tree = []
        X_oob = []
        y_oob = []
        used_indexes = []
        for i in range(len(X)):
            n = random.randint(0, len(X) - 1)
            used_indexes.append(n)
            X_tree.append(X[n])
            y_tree.append(y[n])
        # list of all the indexes used
        used_indexes = list(dict.fromkeys(used_indexes))
        for i in range(len(X)):
            # out of bag observations
            if not (i in used_indexes):
                X_oob.append(X[i])
                y_oob.append(y[i])
        return X_tree, y_tree, X_oob, y_oob



    def calculate_weight(self, tree, X_oob, y_oob):
        '''
          This method calculates a weight for the given tree, based on it's performance on
          the OOB instances. We support two different types:
          if self.weight_type == 'div', we should return 1/error and if it's 'sub' we should
          return 1-error. The error is the normalized error rate of the tree on OOB instances.
          For classification use 0/1 loss error (i.e., count 1 for every wrong OOB instance and divide by the numbner of OOB instances),
          and for regression use mean square error of the OOB instances.
        '''
        # calculate the error
        # For classification use 0/1 loss error
        if self.type == "cat":
            # count 1 for every wrong OOB instance and divide by the numbner of OOB instances)
            error = zero_one_loss(tree.predict(X_oob), y_oob)
        # for regression use mean square error of the OOB instances
        else:
            error = mean_squared_error(tree.predict(X_oob), y_oob)

        # calculate weights
        # if self.weight_type == 'div', we should return 1/error
        if self.weight_type == 'div':
            return 1 / error
        # if it's 'sub' we should return 1-error.
        if self.weight_type == 'sub':
            return 1 - error

    def predict(self, X):
        '''
          Predict the label/value of the given instances in the X matrix.
          For classification you should implement weighted voting, and for regression implement weighted mean.
          Return a list of predictions for the given instances.
        '''
        predictions = []
        pred_of_y = []
        # For classification you should implement weighted voting
        if self.type == "cat":
            for tree in self.trees:  # prediction of single tree
                pred_of_y.append(tree.predict(X))
            # count the votes of preditions from trees
            pred_of_y = np.asarray(pred_of_y)
            for i in range(pred_of_y.shape[1]):
                count = np.bincount(pred_of_y[:, i], weights=self.weights)
                prediction = np.argmax(count)
                predictions.append(prediction)
        # for regression implement weighted mean
        else:
            for tree in self.trees:  # prediction of single tree
                pred_of_y.append(tree.predict(X))
            # multiply the prediction in the weight of trees
            pred_of_y = np.asarray(pred_of_y)
            #pred_of_y =  np.linalg.norm(pred_of_y)
            self.weights = self.weights.reshape((self.n_trees, 1))
            predictions = np.multiply(pred_of_y, self.weights)
            # calculate the mean for prediction from all trees
            predictions = np.mean(pred_of_y, axis=0)
        return predictions


# Section 2: Evaluation

In this section you are requested to evaluate your implementation, and compare it with RandomForestClassifier and RandomForestRegressor.

In [ ]:
# TODO 3: Implement a tuning method for your classifier. 
# Note: you could potentially implement WRF as a sklearn classifier and then 
# simply use GridSearchCV inside. For those of you who want to take this route, 
# you are welcome to modify the implementation of WRF accordingly. Check out: https://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator
import itertools as it
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
import pandas as pd




def tune(classifier, X, y, arguments, cv=5):
  '''
    This method is doing exactly what GridSearchCV is doing for a sklearn classifier.
    It will run cross validation training with cv folds many times. Each time it will evaluate the CV "performance" on a different
    combination of the given arguments. You should check every combination of the given arguments and return a dictionary with 
    the best argument combination. For classification, "performance" is accuracy. For Regression, "performance" is mean square error.
    
    classifier: it's the WRF classifier to tune
    X, y: the dataset to tune over
    arguments: a dictionary with keys are one of n_trees, max_depth, n_features, weight_type
    and the values are lists of values to test for each argument (see more in GridSearchCV)
  '''
  
  # <Your code goes here>
  if classifier.type == "cat":
    score = "accuracy"
  else: 
      score = "neg_mean_squared_error"
  gcv = GridSearchCV(classifier, arguments,score ,cv)
  gcv.fit(X, y)
  return gcv.best_params_





In [ ]:
# TODO 4: Evaluate your implementation and compare it to RandomForestClassifier/Regressor provided by sklearn.

# For classification use the Fashion MNIST, but subsample the dataset to contain only 7K instances (out of the 60K available instances, you may simply select the first 7K instance from the data).
# - Tune both classifiers (WRF and RandomForestClassifier) before evaluation.
# - Evaluate both classifiers on the first 5K instances from the provided test data.
# - Report accuracy, and provide a full confusion matrix for each classifier.

# For regression use the California housing dataset from Kaggle that we used in class (available also in the Git repo of the course):
# https://www.kaggle.com/harrywang/housing#housing.csv

# - Split the dataset to train and test (test_size=0.1, random_state=0)
# - Tune both regressors (WRF and RandomForestRegressor) before evaluation on the training set.
# - Evaluate both regressors on the test set.
# - Report mean square error.


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score






Classification

In [ ]:
clsf_data = pd.read_csv('Fashion_DataSet/small_fashion_mnist.csv')
clsf_test_data = pd.read_csv('Fashion_DataSet/fashion-mnist_test.csv')


y = clsf_data.iloc[:, 1].to_numpy()
y_test = (clsf_test_data.iloc[:5000, 0]).to_numpy()
X = clsf_data.iloc[:, 2:].to_numpy()
X_test = clsf_test_data.iloc[:5000, 1:].to_numpy()

print(X.shape)
print(X_test.shape)
print(y_test.shape)


print(y.shape)
# print(y_test.head())



(7000, 784)
(5000, 784)
(5000,)
(7000,)


In [ ]:
wrf = WRF(type='cat')
rf = RandomForestClassifier()

In [ ]:
#Random Forest 
gcv = GridSearchCV(rf, {"max_features": [1, 2],
                        "n_estimators": [5, 7, 9], "max_depth": [3, 4, 5]},scoring="accuracy", cv=5, )
gcv.fit(X, y)
rf_best_params = gcv.best_params_
print("RF best params are:")
print(rf_best_params)

# #Weighted Random Forest 
wrf_arguments = {"n_trees": [5, 7, 9],
                "max_depth": [3, 4, 5], "n_features": [1, 2], "weight_type": ['duv', 'sub']}
# wrf_arguments = {"n_trees": [5],
#                  "max_depth": [3], "n_features": [1], "weight_type": ['duv', 'sub']}
wrf_best_params = tune(wrf, X, y, wrf_arguments, cv=5)
print("\nWRF best params are:")
print(wrf_best_params)


RF best params are:
{'max_depth': 5, 'max_features': 2, 'n_estimators': 9}

WRF best params are:
{'max_depth': 5, 'n_features': 2, 'n_trees': 9, 'weight_type': 'sub'}


In [ ]:
#Weighted Random Forest best params to test
wrf_bestparams_ready = WRF(n_trees=wrf_best_params['n_trees'], max_depth=wrf_best_params['max_depth'],
                n_features=wrf_best_params['n_features'],type='cat', weight_type=wrf_best_params['weight_type'])
wrf_bestparams_ready.fit(X, y)

#Random Forest best params to test
rf_bestparams_ready = RandomForestClassifier(n_estimators=rf_best_params['n_estimators'],
                                  max_depth=rf_best_params['max_depth'],
                                  max_features=rf_best_params['max_features'])
rf_bestparams_ready.fit(X, y)

#Predict the models
wrf_prediction = wrf_bestparams_ready.predict(X_test)
rf_prediction = rf_bestparams_ready.predict(X_test)
print("Classification\n")
print("WRF accuracy: ", accuracy_score(y_test, wrf_prediction))
print("\nRF accuracy: ", accuracy_score(y_test, rf_prediction))
print("Confusion matrix for WRF:\n")
print(confusion_matrix(y_test, wrf_prediction))
print("\n")

print("Confusion matrix for RF:\n")
print(confusion_matrix(y_test, rf_prediction,))

Classification

WRF accuracy:  0.626

RF accuracy:  0.7082
Confusion matrix for WRF:

[[324   5  20 105   1   0  39   1   6   1]
 [  2 451  18  18   0   0   0   1   0   0]
 [ 11   3 428  14   2   5  22   0   4   1]
 [ 19  78  12 387   0   5   4   0   2   0]
 [  2   6 384  91   5   2  10   1   5   0]
 [  0   5  14   2   0 284   0 117   7  61]
 [ 91  12 228 104   0   0  75   1   8   0]
 [  0   0   0   0   0  13   0 415   3  40]
 [  2   8  43  40   2  17  32  12 346   6]
 [  1   2   7   6   0   7   0  56  23 415]]


Confusion matrix for RF:

[[377  12  24  55   4   2  12   0  16   0]
 [  3 447  10  28   1   0   1   0   0   0]
 [ 15   0 362   6  64   1  28   0  14   0]
 [ 43  72   9 367   8   1   6   0   1   0]
 [ 17   7 185  80 161   0  51   0   5   0]
 [  0   0   0   0   0 353   0  88   4  45]
 [133   7  97  42  56   0 165   0  19   0]
 [  0   0   0   0   0  24   0 410   0  37]
 [  1   2  12  12   1   3  12   5 454   6]
 [  1   0   0   0   0   8   0  62   1 445]]


Regression

In [ ]:
#Load regression data
regress_data = pd.read_csv('Fashion_DataSet/housing.csv')
y = regress_data["median_house_value"]
X = regress_data.drop("median_house_value", axis=1)
X = pd.concat([X, pd.get_dummies(['ocean_proximity'], prefix='ocean_proximity')], axis=1)
X =  X.drop(columns=["ocean_proximity"])
X = X.fillna(X.mean())
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1, random_state=0)
X_train = X_train.to_numpy()
X_dev = X_dev.to_numpy()
y_train = y_train.to_numpy()
y_dev = y_dev.to_numpy()








In [ ]:
print("Regression")

wrf = WRF(type='reg')
rf_regression = RandomForestRegressor()
# #Weighted Random Forest 
wrf_arguments = {"n_trees": [5, 7, 9],
                 "max_depth": [3, 4, 5], "n_features": [1, 2], "weight_type": ['duv', 'sub']}
# wrf_arguments = {"n_trees": [5],
#                  "max_depth": [3], "n_features": [1], "weight_type": ['duv', 'sub']}
wrf_best_params = tune(wrf, X_train, y_train, wrf_arguments, cv=5)
print("\nWRF best params are:")
print(wrf_best_params)

#Random Forest 
gcv = GridSearchCV(rf_regression, {"max_features": [1, 2],
                        "n_estimators": [5, 7, 9], "max_depth": [3, 4, 5]},scoring="neg_mean_squared_error", cv=5, )
gcv.fit(X_train, y_train)
rfregression_best_params = gcv.best_params_
print("RF best params are:")
print(rfregression_best_params)

#Weighted Random Forest best params to test
wrf_bestparams_ready = WRF(n_trees=wrf_best_params['n_trees'], max_depth=wrf_best_params['max_depth'],
                n_features=wrf_best_params['n_features'],type='reg', weight_type=wrf_best_params['weight_type'])
wrf_bestparams_ready.fit(X_train, y_train)

#Random Forest best params to test
rf_bestparams_ready = RandomForestClassifier(n_estimators=rfregression_best_params['n_estimators'],
                                  max_depth=rfregression_best_params['max_depth'],
                                  max_features=rfregression_best_params['max_features'])
rf_bestparams_ready.fit(X_train, y_train)
#Predict the models
rf_prediction = rf_bestparams_ready.predict(X_dev)
wrf_prediction = wrf_bestparams_ready.predict(X_dev)

wrf_mean_square_error =  mean_squared_error(y_dev,wrf_prediction)
rf_mean_square_error = mean_squared_error(y_dev, rf_prediction )


print("WRF Mean Square Error:")
print(wrf_mean_square_error)

print("RF Mean Square Error:")
print(rf_mean_square_error)


# I calculated this as well to get values closer to 0-1
wrf_mean_square_log_error =  mean_squared_log_error(y_dev,wrf_prediction)
rf_mean_square_log_error = mean_squared_log_error(y_dev, rf_prediction )

print("WRF Mean Square Log Error:")
print(wrf_mean_square_log_error)

print("RF Mean Square Log Error:")
print(rf_mean_square_log_error)




Regression

WRF best params are:
{'max_depth': 5, 'n_features': 2, 'n_trees': 9, 'weight_type': 'sub'}
RF best params are:
{'max_depth': 5, 'max_features': 2, 'n_estimators': 5}
WRF Mean Square Error:
6443374326.078873
RF Mean Square Error:
88321994918.59496
WRF Mean Square Log Error:
0.17194570275125007
RF Mean Square Log Error:
1.2309941144470435
